In [1]:
import osmnx as ox
import folium
import numpy as np
from shapely.geometry import Point
from geopy.distance import geodesic
import random
from IPython.display import IFrame

In [23]:
# Download the street network of Louisville, KY
place = "Louisville, Kentucky, USA"
graph = ox.graph_from_place(place, network_type='drive')

# Convert the graph to a GeoDataFrame for nodes and edges
nodes, edges = ox.graph_to_gdfs(graph)

In [24]:
#add all parking and gas station tags to the points of interest
pois = ox.geometries_from_place(place, tags={"amenity": ["parking", "fuel"]})

#coordinates
candidate_locations = pois[['geometry']]  

#filter out anytning other than points, because points have the coords
candidate_locations = candidate_locations[candidate_locations.geometry.geom_type == "Point"]

#extract coordinates
candidate_locations['latitude'] = candidate_locations.geometry.y
candidate_locations['longitude'] = candidate_locations.geometry.x

#create a random amount of cost for each location
candidate_locations['cost'] = np.random.randint(1000, 8000, size=len(candidate_locations))

C:\Users\Cmancity\AppData\Local\Temp\ipykernel_192536\632138392.py:2: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  pois = ox.geometries_from_place(place, tags={"amenity": ["parking", "fuel"]})


In [62]:
#constants to be adjusted for the GA
NUM_GENERATIONS = 100
POPULATION_SIZE = 100
MUTATION_RATE = 0.2
COVERAGE_RADIUS = 4
MIN_DISTANCE = 8
BUDGET = 250000  
NUM_CANDIDATES = 87 #candidates has to = chromosome length

In [ ]:
# Fitness Function
def fitness_function(chromosome):
    total_cost = 0
    total_coverage = set()
    selected_locations = []

    # Calculate total cost and check constraints
    for i, gene in enumerate(chromosome):
        if gene == 1:
            total_cost += candidate_locations.iloc[i]["cost"]  # Use the randomized cost
            selected_locations.append((candidate_locations.iloc[i]["latitude"], candidate_locations.iloc[i]["longitude"]))

    if total_cost > BUDGET:
        #print("gone over the budget")
        return 0  

    # Penalize if selected locations are too close to each other
    for i, loc1 in enumerate(selected_locations):
        for j, loc2 in enumerate(selected_locations):
            if i != j and geodesic(loc1, loc2).km < MIN_DISTANCE:
                return 0  # Penalize and discard solution

    # Calculate coverage
    for loc1 in selected_locations:
        for loc2 in candidate_locations.itertuples():
            if geodesic(loc1, (loc2.latitude, loc2.longitude)).km <= COVERAGE_RADIUS:
                total_coverage.add(loc2.Index)

    # Fitness = Coverage - Normalized Cost
    fitness = len(total_coverage) - (total_cost / BUDGET) * 100
    print(f"fitness: {fitness}")
    return fitness

In [61]:
def initialize_population(size, num_candidates):
    population = []
    for _ in range(size):
        while True:
            chromosome = np.random.randint(2, size=num_candidates)
            total_cost = sum(candidate_locations.iloc[idx]["cost"] for idx in np.where(chromosome == 1)[0])
            if total_cost < BUDGET:  # Only accept valid chromosomes
                population.append(chromosome)
                break
    return population

In [8]:
# Selection (Tournament Selection)
def select_parents(population, fitnesses):
    tournament_size = 5
    selected = []
    for _ in range(2):
        competitors = random.sample(range(len(population)), tournament_size)
        best = max(competitors, key=lambda idx: fitnesses[idx])
        selected.append(population[best])
    return selected

In [9]:
# Crossover
def crossover(parents):
    point = random.randint(1, NUM_CANDIDATES - 1)
    return np.concatenate((parents[0][:point], parents[1][point:]))

In [10]:
# Mutation
def mutate(chromosome, mutation_rate):
    for i in range(len(chromosome)):
        if random.random() < mutation_rate:
            chromosome[i] = 1 - chromosome[i]
    return chromosome

In [63]:
# Genetic Algorithm
population = initialize_population(POPULATION_SIZE, NUM_CANDIDATES)

#print(population)

In [64]:
# Calculate fitness for the initial population
fitnesses = [fitness_function(ind) for ind in population]

# Find the chromosome with the highest fitness
initial_best_solution = population[np.argmax(fitnesses)]
print(initial_best_solution)

# Create a base map centered on the approximate middle of candidate locations
center_lat = candidate_locations["latitude"].mean()
center_lon = candidate_locations["longitude"].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Add all candidate locations as blue markers
for _, row in candidate_locations.iterrows():
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=5,
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.6,
        popup="Candidate Location"
    ).add_to(m)
    # Add a circle to represent coverage area
    folium.Circle(
        location=[row["latitude"], row["longitude"]],
        radius=COVERAGE_RADIUS * 1000,  # Convert km to meters
        color="green",
        fill=True,
        fill_color="green",
        fill_opacity=0.2
    ).add_to(m)

# Add selected charging stations from the initial best solution as green markers
for idx in np.where(initial_best_solution == 1)[0]:
    row = candidate_locations.iloc[idx]
    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        icon=folium.Icon(color="green", icon="charging-station", prefix="fa"),
        popup="Initial Best Charging Station"
    ).add_to(m)

# Save the map to an HTML file
m.save('initial_best_solution.html')

# Optionally display the map in Jupyter Notebook
IFrame('initial_best_solution.html', width=800, height=600)

[0 0 1 1 1 1 0 1 1 0 1 1 1 1 1 1 0 0 1 0 1 0 0 0 1 0 1 0 1 0 1 1 0 1 0 1 1
 1 1 1 0 1 1 0 1 0 1 1 1 0 1 1 0 1 1 0 1 0 0 1 1 0 0 1 1 1 1 1 0 1 0 1 1 1
 0 1 1 1 1 0 1 0 1 0 0 1 0]


In [65]:
# Genetic Algorithm
for generation in range(NUM_GENERATIONS):
    print(generation)
    fitnesses = [fitness_function(ind) for ind in population]
    new_population = []

    for _ in range(POPULATION_SIZE // 2):
        parents = select_parents(population, fitnesses)
        offspring1 = crossover(parents)
        offspring2 = crossover(parents)
        new_population.extend([mutate(offspring1, MUTATION_RATE), mutate(offspring2, MUTATION_RATE)])

    population = new_population

# Select Best Solution
fitnesses = [fitness_function(ind) for ind in population]
best_solution = population[np.argmax(fitnesses)]

# Output Best Solution
selected_locations = candidate_locations.iloc[np.where(best_solution == 1)[0]]
print("Selected Charging Stations:")
print(selected_locations)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
Selected Charging Stations:
                                            geometry   latitude  longitude  \
element_type osmid                                                           
node         164667181    POINT (-85.73231 38.23628)  38.236278 -85.732311   
             1787639672   POINT (-85.77539 38.08455)  38.084548 -85.775392   
             1787761426   POINT (-85.84296 38.05742)  38.057419 -85.842963   
             1829284744   POINT (-85.69776 38.27639)  38.276395 -85.697761   
             3453183885   POINT (-85.74173 38.24885)  38.248853 -85.741730   
             3454788759   POINT (-85.61450 38.13822)  38.138221 -85.614496   
             3468773442   POINT (-85.88900 38.13996)  38.1

In [66]:
# Create a base map centered on the approximate middle of your candidate locations
center_lat = candidate_locations["latitude"].mean()
center_lon = candidate_locations["longitude"].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

#print(fitnesses)

# Add all candidate locations as blue markers
for _, row in candidate_locations.iterrows():
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=5,
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.6,
        popup="Candidate Location"
    ).add_to(m)

# Add selected charging stations as green markers
for idx in np.where(best_solution == 1)[0]:
    row = candidate_locations.iloc[idx]
    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        icon=folium.Icon(color="green", icon="charging-station", prefix="fa"),
        popup="Selected Charging Station"
    ).add_to(m)
    # Add a circle to represent coverage area
    folium.Circle(
        location=[row["latitude"], row["longitude"]],
        radius=COVERAGE_RADIUS * 1000,  # Convert km to meters
        color="green",
        fill=True,
        fill_color="green",
        fill_opacity=0.2
    ).add_to(m)

# Save the map to an HTML file
m.save('charging_stations.html')

IFrame('charging_stations.html', width=800, height=600)

In [67]:
# Calculate the total cost of the final solution
total_cost = sum(
    candidate_locations.iloc[idx]["cost"] for idx in np.where(best_solution == 1)[0]
)

# Print the total cost
print(f"Total Cost of the Final Solution: ${total_cost}")

Total Cost of the Final Solution: $240430


In [60]:
for generation in range(NUM_GENERATIONS):
    fitnesses = [fitness_function(ind) for ind in population]
    best_idx = np.argmax(fitnesses)
    best_solution = population[best_idx]
    total_cost = sum(candidate_locations.iloc[idx]["cost"] for idx in np.where(best_solution == 1)[0])
    print(f"Generation {generation}: Best Fitness = {fitnesses[best_idx]}, Total Cost = {total_cost}")

Generation 0: Best Fitness = 0, Total Cost = 216628
Generation 1: Best Fitness = 0, Total Cost = 216628
Generation 2: Best Fitness = 0, Total Cost = 216628
Generation 3: Best Fitness = 0, Total Cost = 216628
Generation 4: Best Fitness = 0, Total Cost = 216628
Generation 5: Best Fitness = 0, Total Cost = 216628


KeyboardInterrupt: 